In [2]:
import numpy as np
import sympy as sym
import control as ctrl
from matplotlib import pyplot as plt

In [3]:
g=9.81  # m/s^2
L=0.125 # distance from pendulum's centre of mass to pivot in m
a=0.016 # radius of pulley in m
m=0.32  # mass of pendulum in kg
M=0.7  # mass of carriage in kg
I=8e-5  # moment of inertia on motor shaft in kg m^2
km = 0.08  # torque motor constant in Nm/A
ka = -0.50 # amplifier constant in A/V

gamma =  M/m + I/(m*np.power(a,2))

In [4]:

omega_1 = sym.symbols('omega_1')
omega_0 = sym.symbols('omega_0')

Ac = sym.Matrix(4,4, [0,1,0,0,
                        0,0,omega_1**2 - omega_0**2, 0,
                        0,0,0,1,
                        0,0,-omega_0**2, 0])

B = sym.Matrix(4,1, [0,1,0,1])
Sc=np.diag([-1/12.5, -1/2.23, L/3.18, L/0.64])
opamp_c = np.diag([-20,-30, 20, -10]) # for crane controller
Cc=-(ka*km/(m*a*gamma))*np.linalg.solve(Sc,opamp_c)

s = sym.symbols('s')
t = sym.symbols('t')

k1,k2,k3,k4 = sym.symbols('k1 k2 k3 k4')

K =  sym.Matrix(1,4, [k1,k2,k3,k4])

cG_ = sym.eye(4)*s - Ac
Gc_sys = Cc * cG_.inv() * B

cG_ = sym.eye(4)*s - Ac + B*K
crane_poly = sym.poly(sym.det(cG_), s)
Gc_closed = Cc * cG_.inv() * B

#xc = sympy.inverse_laplace_transform(Gc / s, s, t)

# factorise and convert to latex
print("Closed loop characteristic polynomial for crane model:")
print(sym.latex(crane_poly))

print("Closed loop transfer function for crane model:")
print(sym.latex(sym.simplify(Gc_closed)))

Closed loop characteristic polynomial for crane model:
\operatorname{Poly}{\left( s^{4} + \left(k_{2} + k_{4}\right) s^{3} + \left(k_{1} + k_{3} + \omega_{0}^{2}\right) s^{2} + k_{2} \omega_{1}^{2} s + k_{1} \omega_{1}^{2}, s, domain=\mathbb{Z}\left[k_{1}, k_{2}, k_{3}, k_{4}, \omega_{0}, \omega_{1}\right] \right)}
Closed loop transfer function for crane model:
\left[\begin{matrix}\frac{617.283950617284 \left(\omega_{1}^{2} + s^{2}\right)}{k_{1} \omega_{1}^{2} + k_{1} s^{2} + k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} + \omega_{0}^{2} s^{2} + s^{4}}\\\frac{165.185185185185 s \left(\omega_{1}^{2} + s^{2}\right)}{k_{1} \omega_{1}^{2} + k_{1} s^{2} + k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} + \omega_{0}^{2} s^{2} + s^{4}}\\\frac{1256.2962962963 s^{2}}{k_{1} \omega_{1}^{2} + k_{1} s^{2} + k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} + \omega_{0}^{2} s^{2} + s^{4}}\\- \frac{126.41975308642 s^{3}}{k_{1} \omega_{1}^{2} + k_{1} 

for n = 4:
$a_i > 0 , \quad a_1a_2a_3 > a_0a_3^2 + a_4a_1^2$

In [5]:
# get routh hourwitz conditions

#print(dir(crane_poly))

crane_coeffs = crane_poly.all_coeffs()

for i in range(4):
    print(sym.latex(crane_coeffs[i]), "> 0")

crane_left_inequality = crane_coeffs[1] * crane_coeffs[2] * crane_coeffs[3]
crane_right_inequality = crane_coeffs[0] * crane_coeffs[3] ** 2 + crane_coeffs[4] * crane_coeffs[1] ** 2
crane_inequality_expr = crane_left_inequality.expand() - crane_right_inequality.expand()

crane_inequality_quadratic = sym.poly(crane_inequality_expr / omega_1**2, k2)

print(sym.latex(crane_inequality_quadratic))

crane_k2 = sym.solve(crane_inequality_quadratic, k2)[0] # only 0 satisfies previous RH criteria

print("LaTeX Solutions for k2 inequality")
print(sym.latex(crane_k2))

# now compute poles from k2



1 > 0
k_{2} + k_{4} > 0
k_{1} + k_{3} + \omega_{0}^{2} > 0
k_{2} \omega_{1}^{2} > 0
\operatorname{Poly}{\left( \left(k_{3} + \omega_{0}^{2} - \omega_{1}^{2}\right) k_{2}^{2} + \left(- k_{1} k_{4} + k_{3} k_{4} + k_{4} \omega_{0}^{2}\right) k_{2} -  k_{1} k_{4}^{2}, k_{2}, domain=\mathbb{Z}\left(k_{1}, k_{3}, k_{4}, \omega_{0}, \omega_{1}\right) \right)}
LaTeX Solutions for k2 inequality
\frac{k_{4} \left(k_{1} - k_{3} - \omega_{0}^{2} - \sqrt{k_{1}^{2} + 2 k_{1} k_{3} + 2 k_{1} \omega_{0}^{2} - 4 k_{1} \omega_{1}^{2} + k_{3}^{2} + 2 k_{3} \omega_{0}^{2} + \omega_{0}^{4}}\right)}{2 \left(k_{3} + \omega_{0}^{2} - \omega_{1}^{2}\right)}


In [6]:
# at limit of stability the real part of s is 0
b = sym.symbols("b")

print(crane_poly)
# now solve for b

b_vals = sym.solve(crane_poly.subs({s : 0 + 1j*b}), b)

print(b_vals)


Poly(s**4 + (k2 + k4)*s**3 + (k1 + k3 + omega_0**2)*s**2 + k2*omega_1**2*s + k1*omega_1**2, s, domain='ZZ[k1,k2,k3,k4,omega_0,omega_1]')
[Piecewise((0.25*I*k2 + 0.25*I*k4 - 0.588795921500241*sqrt(0.480749856769136*k1 + 0.480749856769136*k3 + 0.480749856769136*omega_0**2 + 0.180281196288426*(-I*k2 - I*k4)**2 - (-0.0277777777777778*(-k1 - k3 - omega_0**2 - 0.375*(-I*k2 - I*k4)**2)**3 + (-k1 - k3 - omega_0**2 - 0.375*(-I*k2 - I*k4)**2)*(0.0625*k1*k2**2 + 0.125*k1*k2*k4 + 0.0625*k1*k4**2 + k1*omega_1**2 - 0.01171875*k2**4 - 0.046875*k2**3*k4 + 0.0625*k2**2*k3 - 0.0703125*k2**2*k4**2 + 0.0625*k2**2*omega_0**2 - 0.25*k2**2*omega_1**2 + 0.125*k2*k3*k4 - 0.046875*k2*k4**3 + 0.125*k2*k4*omega_0**2 - 0.25*k2*k4*omega_1**2 + 0.0625*k3*k4**2 - 0.01171875*k4**4 + 0.0625*k4**2*omega_0**2) - 0.375*(-0.5*I*k1*k2 - 0.5*I*k1*k4 + 0.125*I*k2**3 + 0.375*I*k2**2*k4 - 0.5*I*k2*k3 + 0.375*I*k2*k4**2 - 0.5*I*k2*omega_0**2 + I*k2*omega_1**2 - 0.5*I*k3*k4 + 0.125*I*k4**3 - 0.5*I*k4*omega_0**2)**2)**0.3333333333